In [ ]:
import sys
import os

# Add the path to the project directory
utils_path = os.path.abspath('..')
if utils_path not in sys.path:
    sys.path.append(utils_path)
# Add the path to the directory containing utils to sys.path
utils_path = os.path.abspath('../utils')
if utils_path not in sys.path:
    sys.path.append(utils_path)
print(sys.path)

### Data Preprocessing and Visualization

Let's start by loading the dataset and visualizing random images with bounding boxes.



In [ ]:
import os
import pandas as pd
import matplotlib.pyplot as plt

# Read the dataset
# data = pd.read_excel('WRB_trimmed.xlsx')
data = pd.read_excel('../dataset/WRB_All.xlsx')

# Split the data into training and test sets based on the 'Group Name' column
train_data = data[data['Group Name'] == 'Train']
test_data = data[data['Group Name'] == 'Test']

### Authenticate the client

Instantiate a training and prediction client with your endpoint and keys. 

In [2]:
from azure.cognitiveservices.vision.customvision.training import CustomVisionTrainingClient
from azure.cognitiveservices.vision.customvision.prediction import CustomVisionPredictionClient
from msrest.authentication import ApiKeyCredentials
from dotenv import load_dotenv

# Load environment variables
load_dotenv()
print(os.getenv("TRAINING_KEY"), os.getenv("TRAINING_ENDPOINT"))

# Authentication
credentials = ApiKeyCredentials(in_headers={"Training-key": os.getenv("TRAINING_KEY")})
trainer = CustomVisionTrainingClient(endpoint=os.getenv("TRAINING_ENDPOINT"), credentials=credentials)

# Authentication for prediction
prediction_credentials = ApiKeyCredentials(in_headers={"Prediction-key": os.getenv("PREDICTION_KEY")})
predictor = CustomVisionPredictionClient(endpoint=os.getenv("PREDICTION_ENDPOINT"), credentials=prediction_credentials)


90dad624b6664556accbcfd69e2e170d https://crackdetection.cognitiveservices.azure.com/


### Creat or get the project

In [3]:
# Find the object detection domain
obj_detection_domain = next(domain for domain in trainer.get_domains() if domain.type == "ObjectDetection" and domain.name == "General")

# Project name setup
project_name = "WRB-Bad-Detection"

# Find project by name
print(f"Searching for project '{project_name}'...")
projects = trainer.get_projects()

project_id = None
for project in projects:
    if project.name == project_name:
        project_id = project.id
        break

if project_id:
    print(f"Project '{project_name}' found with ID: {project_id}")
else:
    print(f"No project found with the name '{project_name}'")
    # Create a new project
    print ("Creating project...")
    project = trainer.create_project(project_name, domain_id=obj_detection_domain.id)


Searching for project 'WRB-Bad-Detection'...
Project 'WRB-Bad-Detection' found with ID: 1ee0bb48-3b3f-419d-a575-e12c98f91578


### Data Ingestion and Transformation

Create a data pipeline to transform and ingest data according to Azure Custom Vision's requirements.

In [15]:
from azure.cognitiveservices.vision.customvision.training.models import Region, ImageUrlCreateEntry, ImageUrlCreateBatch
from tqdm import tqdm
import json
from utils.transform_functions import polygon_to_bbox

# Create tags in Custom Vision
tag_dict = {}
for label in data['Label'].unique():
    tag_dict[label] = trainer.create_tag(project.id, label)

# Convert polygons to bounding boxes
train_data['bbox'] = train_data['2D Image Points'].apply(json.loads).apply(polygon_to_bbox)

def upload_images_to_azure(data, trainer, project_id):

    for index, row in tqdm(data.iterrows(), total=data.shape[0]):
        img_width = 1000
        img_height = 750
        bbox = row['bbox']
        label = row['Label']

        regions = [Region(
            tag_id=tag_dict[label].id,
            left=bbox["left"] / img_width,
            top=bbox["top"] / img_height,
            width=bbox["width"] / img_width,
            height=bbox["height"] / img_height
        )]

        image_entry = ImageUrlCreateEntry(
            url=row['2D Image URL'],
            regions=regions
        )

        upload_result = trainer.create_images_from_urls(
            project_id, 
            ImageUrlCreateBatch(images=[image_entry])
            )

        # # If upload is not successful, print details about that image for debugging
        # if not upload_result.is_batch_successful:
        #     print("Image upload failed.")
        #     for image in upload_result.images:
        #         print(f"Image status: {image.status}, Source URL: {image.source_url}")
        #     break
        # else:
        #     print(f"Uploaded image: {row['2D Image URL']}")


upload_images_to_azure(train_data, trainer, project.id)


90dad624b6664556accbcfd69e2e170d https://crackdetection.cognitiveservices.azure.com/
Creating project...


C:\Users\User\AppData\Local\Temp\ipykernel_21920\1436926017.py:51: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_data['bbox'] = train_data['2D Image Points'].apply(json.loads).apply(polygon_to_bbox)
100%|██████████| 3093/3093 [1:02:25<00:00,  1.21s/it]
